# Third notebook - upload file df_final_last_GitHub

In [ ]:
import pandas as pd

df = pd.read_csv('df_final_last_GitHub.csv')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Start with ydata-profiling report - to have a good overview of all data set


In [ ]:

!pip install ydata-profiling --quiet

from ydata_profiling import ProfileReport
import pandas as pd


In [ ]:



# create report
profile = ProfileReport(df, title="EDA Report - Dataset Overview", explorative=True)

profile.to_notebook_iframe()

# Save HTML file
profile.to_file("EDA_report.html")

print("✅ דו\"ח EDA נוצר ונשמר כ-'EDA_report.html'")


In [ ]:
profile.to_file("EDA_report.html")


In [ ]:

from google.colab import files
files.download("EDA_report.html")


In [ ]:
df.head()

# Explore target - popularity

In [ ]:
# See 'popularity' values


unique_values = df['popularity'].unique()

# how many values
print("Number of unique values:", len(unique_values))
print(unique_values)

# five first values
print(unique_values[:5])

# amount-count
print(df['popularity'].value_counts())


In [ ]:
# lowest 5 values
lowest_5 = np.sort(df['popularity'].unique())[:5]
print(lowest_5)

In [ ]:
df['popularity'].describe()

In [ ]:
# See 'popularity' as a Log transformation

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# -----------------------------
# transformation-log
# -----------------------------
df['popularity_log'] = np.log1p(df['popularity'])

# -----------------------------
# Boxplot
# -----------------------------
plt.figure(figsize=(14, 4))
sns.boxplot(x='popularity_log', data=df, color='#66c2a5')
plt.title("Distribution of log(Popularity) - Before Categorization")
plt.xlabel("log(1 + Popularity)")
plt.tight_layout()
plt.show()

# -----------------------------
# Histogram
# -----------------------------
plt.figure(figsize=(16, 5))
sns.histplot(df['popularity_log'], bins=30, kde=True, color='#66c2a5')
plt.title("Histogram of log(Popularity) - Before Categorization")
plt.xlabel("log(1 + Popularity)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


## Divide 'popularity' into three main categories - low, med and hit - this will be the target analysis

In [ ]:
# Divide 'popularity' into three main categories - low, med and hit
# Moreover - balance the category to fit ML models predication

import pandas as pd
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# create three categories
# -----------------------------
labels = ['Low', 'Mid', 'Hit']
df['popularity_3cat'] = pd.qcut(df['popularity'], q=3, labels=labels)

# -----------------------------
# check
# -----------------------------
print("Initial distribution per category:")
print(df['popularity_3cat'].value_counts())
print("\nAverage popularity per category:")
print(df.groupby('popularity_3cat')['popularity'].mean())

# -----------------------------
# create graphs
# -----------------------------
plt.figure(figsize=(10, 6))
sns.boxplot(x='popularity_3cat', y='popularity', data=df, palette='Set2')
plt.title("Popularity Distribution per Category (Before Balancing)")
plt.xlabel("Popularity Category")
plt.ylabel("Popularity")
plt.show()

plt.figure(figsize=(8, 5))
sns.countplot(x='popularity_3cat', data=df, palette='Set2')
plt.title("Number of Samples per Category (Before Balancing)")
plt.xlabel("Popularity Category")
plt.ylabel("Count")
plt.show()

# -----------------------------
# Split by categories
# -----------------------------
df_low = df[df['popularity_3cat'] == 'Low']
df_mid = df[df['popularity_3cat'] == 'Mid']
df_hit = df[df['popularity_3cat'] == 'Hit']

# -----------------------------
# Increase the Mid category to the size of the largest category
# -----------------------------
target_count = max(len(df_low), len(df_hit))
df_mid_upsampled = resample(
    df_mid,
    replace=True,       # מאפשר שכפול
    n_samples=target_count,
    random_state=42
)

# -----------------------------
# all data
# -----------------------------
df_balanced = pd.concat([df_low, df_mid_upsampled, df_hit])

# -----------------------------
# Check after balancing
# -----------------------------
print("\nBalanced distribution per category:")
print(df_balanced['popularity_3cat'].value_counts())

print("\nAverage popularity per category after balancing:")
print(df_balanced.groupby('popularity_3cat')['popularity'].mean())

# -----------------------------
# graphs after balancing
# -----------------------------
plt.figure(figsize=(10, 6))
sns.boxplot(x='popularity_3cat', y='popularity', data=df_balanced, palette='cool')
plt.title("Popularity Distribution per Category (After Balancing)")
plt.xlabel("Popularity Category")
plt.ylabel("Popularity")
plt.show()

plt.figure(figsize=(8, 5))
sns.countplot(x='popularity_3cat', data=df_balanced, palette='cool')
plt.title("Number of Samples per Category (After Balancing)")
plt.xlabel("Popularity Category")
plt.ylabel("Count")
plt.show()


In [ ]:
# More figures for the transformation

import pandas as pd
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# -----------------------------
# create three categories
# -----------------------------
labels = ['Low', 'Mid', 'Hit']
df['popularity_3cat'] = pd.qcut(df['popularity'], q=3, labels=labels)

# -----------------------------
# transformation - log
# -----------------------------
df['popularity_log'] = np.log1p(df['popularity'])  # log(1 + popularity)

# -----------------------------
# Split by categories
# -----------------------------
df_low = df[df['popularity_3cat'] == 'Low']
df_mid = df[df['popularity_3cat'] == 'Mid']
df_hit = df[df['popularity_3cat'] == 'Hit']

# -----------------------------
# Increase the Mid category to the size of the largest category
# -----------------------------
target_count = max(len(df_low), len(df_hit))
df_mid_upsampled = resample(
    df_mid,
    replace=True,
    n_samples=target_count,
    random_state=42
)

# -----------------------------
# Connecting all the data
# -----------------------------
df_balanced = pd.concat([df_low, df_mid_upsampled, df_hit])

# -----------------------------
# Graphs after log transformation and balancing
# -----------------------------

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(y='popularity_3cat', x='popularity_log', data=df_balanced, palette='viridis')
plt.title("Log(Popularity) Distribution per Category (After Balancing)")
plt.ylabel("Popularity Category")
plt.xlabel("log(1 + Popularity)")
plt.tight_layout()
plt.show()

# Barplot
plt.figure(figsize=(8, 5))
sns.barplot(
    y='popularity_3cat',
    x='popularity_log',
    data=df_balanced,
    estimator=np.mean,
    ci='sd',
    palette='viridis'
)
plt.title("Mean log(Popularity) per Category (After Balancing)")
plt.ylabel("Popularity Category")
plt.xlabel("Mean log(1 + Popularity)")
plt.tight_layout()
plt.show()


In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# To have an overview of missing data and types of data

summary = pd.DataFrame({
    'non_null_count': df.notnull().sum(),
    'null_percent': df.isnull().mean() * 100,
    'num_unique': df.nunique(),
    'data_type': df.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)


# Before continue - remove unneeded columns
# Imputetion - Fill nulls in columns 'first_year' and 'last_year'

In [ ]:
# Drop columns that are not useful for modeling
df = df.drop(columns=['final_name', 'id', 'vote_count', 'vote_average', 'type', 'overview', 'episode_run_time_outlier_iqr', 'popularity_log'])

print("Remaining columns after dropping 'final_name' and 'id':")
print(df.columns)



In [ ]:
# Imputetion for 'first_year' and 'last_year' by using other relevant columns

# Columns that have a logical relationship to years
cols_for_impute = ['production_countries', 'networks', 'original_language', 'type_encoded']

# -----------------------------
# by Fill first_year
# -----------------------------
df['first_year'] = df.groupby(cols_for_impute)['first_year'].transform(
    lambda x: x.fillna(x.mean())
)

# -----------------------------
# by Fill last_year
# -----------------------------
df['last_year'] = df.groupby(cols_for_impute)['last_year'].transform(
    lambda x: x.fillna(x.mean())
)

# -----------------------------
# Fallback: If there is still NaN, we will use the overall average of the column
# -----------------------------
df['first_year'] = df['first_year'].fillna(df['first_year'].mean())
df['last_year'] = df['last_year'].fillna(df['last_year'].mean())

# -----------------------------
# final check
# -----------------------------
print(df[['first_year','last_year']].isnull().sum())
print(df[['first_year','last_year']].describe())


In [ ]:
df.info()

In [ ]:
df.head()

# Before the next step - more correlation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a series with correlation against popularity
correlations = df.select_dtypes(include=['number', 'bool']).corrwith(df['popularity'])

# remove 'popularity'
correlations = correlations.drop(['popularity'], errors='ignore')

# Sort by correlation strength (highest to lowest)
correlations = correlations.sort_values(ascending=False)

# create DataFrame
corr_table = correlations.reset_index()
corr_table.columns = ['Feature', 'Correlation_with_popularity']

top30 = corr_table.head(30)
top30 = top30.iloc[::-1]


plt.figure(figsize=(10, 6))
sns.barplot(
    x='Correlation_with_popularity',
    y='Feature',
    data=top30,
    color='skyblue'
)

# add values
for i, value in enumerate(top30['Correlation_with_popularity']):
    plt.text(value + 0.002, i, f"{value:.3f}", va='center', fontsize=10, color='navy')

plt.title("Top 30 Correlations with Popularity", fontsize=14)
plt.xlabel("Pearson correlation (r)")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


# Create a series with correlation against popularity

correlations = df.select_dtypes(include=['number', 'bool']).corrwith(df['popularity'])

# Filter both 'popularity' and 'popularity_log'
correlations = correlations.drop(['popularity'], errors='ignore')

# Keep only positive values ​​(negatives become zero)
correlations = correlations.clip(lower=0)

# Sort by correlation strength (highest to lowest)
correlations = correlations.sort_values(ascending=False)

# create DataFrame
corr_table = correlations.reset_index()
corr_table.columns = ['Feature', 'Correlation_with_popularity']

# Horizontal bar graph for all variables
plt.figure(figsize=(12, 8))
sns.barplot(
    x='Correlation_with_popularity',
    y='Feature',
    data=corr_table,
    color='skyblue'
)

# Add correlation values ​​next to each bar
for i, value in enumerate(corr_table['Correlation_with_popularity']):
    plt.text(value + 0.002, i, f"{value:.3f}", va='center', fontsize=10, color='navy')

plt.title("Correlations with Popularity (Negative set to 0) – All Features", fontsize=16)
plt.xlabel("Pearson correlation (r)")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# =========================
# Creating new features
# =========================
df['vote_count_x_vote_avg'] = df['vote_count_group'] * df['vote_average_group']
df['episodes_per_season'] = df['number_of_episodes_clean'] / (df['number_of_seasons_clean'] + 1)
df['age'] = df['last_year'] - df['first_year']

# Handling infinite or missing values
for col in ['vote_count_x_vote_avg', 'episodes_per_season', 'age']:
    df[col] = df[col].replace([np.inf, -np.inf], 0).fillna(0)

# =========================
# Calculating correlation against popularity
# =========================
numeric_cols = df.select_dtypes(include=['number', 'bool']).columns.drop(['popularity'], errors='ignore')
correlations = df[numeric_cols].corrwith(df['popularity'])

# Negative values ​​become zero
correlations = correlations.clip(lower=0)

# Sort by correlation strength (highest to lowest)
correlations = correlations.sort_values(ascending=False)

# Create DataFrame
corr_table = correlations.reset_index()
corr_table.columns = ['Feature', 'Correlation_with_popularity']

# =========================
# Horizontal bar graph
# =========================
plt.figure(figsize=(12, 8))
sns.barplot(
    x='Correlation_with_popularity',
    y='Feature',
    data=corr_table,
    color='skyblue'
)

# Adding correlation values ​​next to each bar
for i, value in enumerate(corr_table['Correlation_with_popularity']):
    plt.text(value + 0.002, i, f"{value:.3f}", va='center', fontsize=10, color='navy')

plt.title("Correlations with Popularity (Negative set to 0) – All Features", fontsize=16)
plt.xlabel("Pearson correlation (r)")
plt.tight_layout()
plt.show()


# Feature selection

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer

# -----------------------------
# Fast sampling if many rows
# -----------------------------
max_rows = 30000
if df.shape[0] > max_rows:
    df_sample = df.sample(max_rows, random_state=42).copy()
else:
    df_sample = df.copy()

# -----------------------------
# Features (all columns except popularity-related)
# -----------------------------
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df_sample.columns if c not in exclude_cols]
X = df_sample[feature_columns]
y = df_sample['popularity_3cat']

# -----------------------------
# Subset features if more than 100
# -----------------------------
max_features = 100
if X.shape[1] > max_features:
    X_fast = X.iloc[:, :max_features].copy()
else:
    X_fast = X.copy()
y_fast = y.copy()

# -----------------------------
# Fill missing values עם mean
# -----------------------------
imputer = SimpleImputer(strategy='mean')
X_fast = pd.DataFrame(imputer.fit_transform(X_fast), columns=X_fast.columns)

# -----------------------------
# Fast linear models
# -----------------------------
logreg_l1 = LogisticRegression(penalty='l1', solver='saga', max_iter=500).fit(X_fast, y_fast)
logreg_l2 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=500).fit(X_fast, y_fast)
svc = LinearSVC(max_iter=500).fit(X_fast, y_fast)

# -----------------------------
# fast Tree-based models
# -----------------------------
rf = RandomForestClassifier(n_estimators=10, max_depth=3, n_jobs=-1).fit(X_fast, y_fast)
gb = GradientBoostingClassifier(n_estimators=10, max_depth=2).fit(X_fast, y_fast)

# -----------------------------
# Feature selection flags
# -----------------------------
logreg_l1_selected = (np.abs(logreg_l1.coef_).sum(axis=0) > 0).astype(int)
logreg_l2_selected = (np.abs(logreg_l2.coef_).sum(axis=0) > 0).astype(int)
svc_selected = (np.abs(svc.coef_).sum(axis=0) > 0).astype(int)
rf_selected = (rf.feature_importances_ > 0).astype(int)
gb_selected = (gb.feature_importances_ > 0).astype(int)

# -----------------------------
# Summary DataFrame
# -----------------------------
selection_df = pd.DataFrame({
    'Feature': X_fast.columns,
    'Logistic_L1': logreg_l1_selected,
    'Logistic_L2': logreg_l2_selected,
    'LinearSVC': svc_selected,
    'RandomForest': rf_selected,
    'GradientBoost': gb_selected
})

selection_df['Sum'] = selection_df[['Logistic_L1','Logistic_L2','LinearSVC','RandomForest','GradientBoost']].sum(axis=1)
selection_df = selection_df.sort_values(by='Sum', ascending=False).reset_index(drop=True)

print(selection_df)


# Model selection

In [ ]:
# Simple models run - no feature engineering
# =====================================
# Simple Models Run + Summary Table + Visualization
# =====================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, log_loss, roc_auc_score

# ===============================
# Creating popular categories
# ===============================
labels = ['Low', 'Mid', 'Hit']
df['popularity_3cat'] = pd.qcut(df['popularity'], q=3, labels=labels)

# ===============================
# Features & target
# ===============================
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df.columns if c not in exclude_cols]
X = df[feature_columns]
y = df['popularity_3cat']

# Fill missing numeric values only
numeric_cols = X.select_dtypes(include=[np.number]).columns
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].mean())

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# ===============================
# Define classifiers
# ===============================
models = {
    'Logistic_L1': LogisticRegression(penalty='l1', solver='saga', max_iter=500, class_weight='balanced'),
    'Logistic_L2': LogisticRegression(penalty='l2', solver='lbfgs', max_iter=500, class_weight='balanced'),
    'LinearSVC': LinearSVC(max_iter=500, class_weight='balanced'),
    'RandomForest': RandomForestClassifier(n_estimators=100, max_depth=5, n_jobs=-1, random_state=42, class_weight='balanced'),
    'GradientBoost': GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'XGBoost': xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(le.classes_),
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric='mlogloss',
        random_state=42,
        n_jobs=-1
    )
}

# ===============================
# Train models & evaluate
# ===============================
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #  XGBoost, AdaBoost, GradientBoost  predict_proba
    try:
        y_proba = model.predict_proba(X_test)
        logloss = log_loss(y_test, y_proba)
        auc = roc_auc_score(pd.get_dummies(y_test), y_proba, multi_class='ovr')
    except:
        #  LinearSVC  predict_proba
        logloss = np.nan
        auc = np.nan

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    results.append({
        'Model': name,
        'Accuracy': round(report['accuracy'], 3),
        'Precision': round(report['macro avg']['precision'], 3),
        'Recall': round(report['macro avg']['recall'], 3),
        'F1-score': round(report['macro avg']['f1-score'], 3),
        'Log-loss': None if np.isnan(logloss) else round(logloss, 3),
        'AUC': None if np.isnan(auc) else round(auc, 3)
    })

    print(f"\n=== {name} ===")
    print("Accuracy:", report['accuracy'])
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# ===============================
# Summary Table
# ===============================
results_df = pd.DataFrame(results)
print("\n=== Summary Table ===")
print(results_df)

# ===============================
# Visualization
# ===============================
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-score']

plt.figure(figsize=(12, 6))
results_melted = results_df.melt(id_vars='Model', value_vars=metrics_to_plot,
                                 var_name='Metric', value_name='Score')
sns.barplot(data=results_melted, x='Metric', y='Score', hue='Model')
plt.title('Model Performance Comparison (Simple Models)', fontsize=14, weight='bold')
plt.ylim(0, 1)
plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


# models with feature engineering

In [ ]:
# A bit improved model - with feature engineering
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix

# ===============================
# Feature Engineering
# ===============================
df['vote_count_x_vote_avg'] = df['vote_count_group'] * df['vote_average_group']
df['episodes_per_season'] = df['number_of_episodes_clean'] / (df['number_of_seasons_clean'] + 1)
df['vote_count_x_vote_avg'] = df['vote_count_x_vote_avg'].replace([np.inf, -np.inf], 0).fillna(0)
df['episodes_per_season'] = df['episodes_per_season'].replace([np.inf, -np.inf], 0).fillna(0)

# ===============================
# Creating popular categories
# ===============================
labels = ['Low', 'Mid', 'Hit']
df['popularity_3cat'] = pd.qcut(df['popularity'], q=3, labels=labels)

# ===============================
# Mid category balance
# ===============================
df_low = df[df['popularity_3cat'] == 'Low']
df_mid = df[df['popularity_3cat'] == 'Mid']
df_hit = df[df['popularity_3cat'] == 'Hit']

target_count = max(len(df_low), len(df_hit))
df_mid_upsampled = resample(df_mid, replace=True, n_samples=target_count, random_state=42)
df_balanced = pd.concat([df_low, df_mid_upsampled, df_hit])

print("Balanced distribution per category:")
print(df_balanced['popularity_3cat'].value_counts())
print("\nAverage popularity per category after balancing:")
print(df_balanced.groupby('popularity_3cat')['popularity'].mean())

# ===============================
# Features & target
# ===============================
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df_balanced.columns if c not in exclude_cols]
X = df_balanced[feature_columns]
y = df_balanced['popularity_3cat']

# Fill missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# ===============================
# Define classifiers
# ===============================
models = {
    'Logistic_L1': LogisticRegression(penalty='l1', solver='saga', max_iter=500, class_weight='balanced'),
    'Logistic_L2': LogisticRegression(penalty='l2', solver='lbfgs', max_iter=500, class_weight='balanced'),
    'LinearSVC': LinearSVC(max_iter=500, class_weight='balanced'),
    'RandomForest': RandomForestClassifier(n_estimators=100, max_depth=5, n_jobs=-1, random_state=42, class_weight='balanced'),
    'GradientBoost': GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'XGBoost': xgb.XGBClassifier(objective='multi:softmax', num_class=len(le.classes_), n_estimators=100,
                                 max_depth=3, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss',
                                 random_state=42, n_jobs=-1)
}

# ===============================
# Train models & evaluate
# ===============================
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"\n=== {name} ===")
    print("Accuracy:", model.score(X_test, y_test))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


In [ ]:
# A very good model - but before Grid optimization
# There is a better model
#
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix

# ==============================
# Assume df_balanced is ready (after upsampling Mid)
# ==============================

# -----------------------------
# Features (exclude popularity-related)
# -----------------------------
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df_balanced.columns if c not in exclude_cols]
X = df_balanced[feature_columns]
y = df_balanced['popularity_3cat']

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Fill missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# -----------------------------
# RandomForest GridSearch
# -----------------------------
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'class_weight': ['balanced'],
    'min_samples_split': [2, 5]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
rf_grid.fit(X_train, y_train)

print("=== Best RandomForest ===")
print("Best params:", rf_grid.best_params_)
y_pred_rf = rf_grid.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# -----------------------------
# XGBoost GridSearch
# -----------------------------
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

xgb_grid = GridSearchCV(
    xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=len(le.classes_),
        use_label_encoder=False,
        eval_metric='mlogloss',
        n_jobs=-1,
        random_state=42
    ),
    xgb_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
xgb_grid.fit(X_train, y_train)

print("\n=== Best XGBoost ===")
print("Best params:", xgb_grid.best_params_)
y_pred_xgb = xgb_grid.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))


In [ ]:
# Feature Importance for the model above

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# best_rf
best_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
best_rf.fit(X_train, y_train)

# -----------------------------
# Feature Importance
# -----------------------------
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

# print 20 most important features
print(feature_importances.head(20))

# -----------------------------
# graph
# -----------------------------
plt.figure(figsize=(10,8))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(20), palette='viridis')
plt.title('Top 20 Feature Importances - RandomForest')
plt.tight_layout()
plt.show()


In [ ]:
# A model with Train / Dev / Test Split
# ===============================
# MODEL: Train / Dev / Test Split
# ===============================
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# ===============================
# Data preparation
# ===============================
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df_balanced.columns if c not in exclude_cols]

X = df_balanced[feature_columns]
y = df_balanced['popularity_3cat']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# ===============================
# Split: Train (60%), Dev (20%), Test (20%)
# ===============================
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

X_train, X_dev, y_train, y_dev = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)  # 0.25 * 0.8 = 0.2

print(f"Train: {len(X_train)}, Dev: {len(X_dev)}, Test: {len(X_test)}")

# ===============================
# Random Forest Grid Search (on Train only)
# ===============================
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'class_weight': ['balanced'],
    'min_samples_split': [2, 5]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
rf_grid.fit(X_train, y_train)

print("=== Best RandomForest (Train) ===")
print("Best params:", rf_grid.best_params_)

# Evaluate on Dev Set
y_pred_dev_rf = rf_grid.predict(X_dev)
print("\n--- RandomForest DEV Performance ---")
print(classification_report(y_dev, y_pred_dev_rf, target_names=le.classes_))
print("Confusion Matrix (Dev):")
print(confusion_matrix(y_dev, y_pred_dev_rf))

# Evaluate on Test Set
y_pred_test_rf = rf_grid.predict(X_test)
print("\n--- RandomForest TEST Performance ---")
print(classification_report(y_test, y_pred_test_rf, target_names=le.classes_))
print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_pred_test_rf))

# ===============================
# XGBoost Grid Search (on Train only)
# ===============================
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

xgb_grid = GridSearchCV(
    xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=len(le.classes_),
        use_label_encoder=False,
        eval_metric='mlogloss',
        n_jobs=-1,
        random_state=42
    ),
    xgb_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
xgb_grid.fit(X_train, y_train)

print("\n=== Best XGBoost (Train) ===")
print("Best params:", xgb_grid.best_params_)

# Evaluate on Dev Set
y_pred_dev_xgb = xgb_grid.predict(X_dev)
print("\n--- XGBoost DEV Performance ---")
print(classification_report(y_dev, y_pred_dev_xgb, target_names=le.classes_))
print("Confusion Matrix (Dev):")
print(confusion_matrix(y_dev, y_pred_dev_xgb))

# Evaluate on Test Set
y_pred_test_xgb = xgb_grid.predict(X_test)
print("\n--- XGBoost TEST Performance ---")
print(classification_report(y_test, y_pred_test_xgb, target_names=le.classes_))
print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_pred_test_xgb))


In [ ]:
# =====================================
# Popularity Prediction (3 categories) + Feature Engineering
# =====================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# ==============================
# Assume df_balanced is ready
# ==============================

# ===============================
# 1️⃣ Feature Engineering
# ===============================
df_balanced['vote_count_x_vote_avg'] = df_balanced['vote_count_group'] * df_balanced['vote_average_group']
df_balanced['episodes_per_season'] = df_balanced['number_of_episodes_clean'] / (df_balanced['number_of_seasons_clean'] + 1)
df_balanced['age'] = df_balanced['last_year'] - df_balanced['first_year']

# ===============================
# 2️⃣ Feature Selection
# ===============================
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df_balanced.columns if c not in exclude_cols]
X = df_balanced[feature_columns]
y = df_balanced['popularity_3cat']

# ===============================
# 3️⃣ Encoding & Imputation
# ===============================
le = LabelEncoder()
y_encoded = le.fit_transform(y)

imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# ===============================
# 4️⃣ Train/Test Split
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# ===============================
# 5️⃣ RandomForest GridSearch
# ===============================
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'class_weight': ['balanced'],
    'min_samples_split': [2, 5]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
rf_grid.fit(X_train, y_train)

print("=== Best RandomForest ===")
print("Best params:", rf_grid.best_params_)
y_pred_rf = rf_grid.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# ===============================
# 6️⃣ XGBoost GridSearch
# ===============================
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

xgb_grid = GridSearchCV(
    xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=len(le.classes_),
        use_label_encoder=False,
        eval_metric='mlogloss',
        n_jobs=-1,
        random_state=42
    ),
    xgb_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
xgb_grid.fit(X_train, y_train)

print("\n=== Best XGBoost ===")
print("Best params:", xgb_grid.best_params_)
y_pred_xgb = xgb_grid.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ===============================
# 🎯 Feature Importance – RandomForest
# ===============================
best_rf = rf_grid.best_estimator_

rf_importances = pd.Series(
    best_rf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
rf_importances.head(15).plot(kind='barh')
plt.gca().invert_yaxis()
plt.title("Top 15 Feature Importances - Random Forest", fontsize=14)
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

print("\n=== RandomForest Feature Importance ===")
print(rf_importances.head(20))

# ===============================
# 🎯 Feature Importance – XGBoost
# ===============================
best_xgb = xgb_grid.best_estimator_

xgb_importances = pd.Series(
    best_xgb.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
xgb_importances.head(15).plot(kind='barh', color='orange')
plt.gca().invert_yaxis()
plt.title("Top 15 Feature Importances - XGBoost", fontsize=14)
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

print("\n=== XGBoost Feature Importance ===")
print(xgb_importances.head(20))


In [ ]:
# The best model!

# =====================================
# Popularity Prediction (3 categories) + Feature Engineering + Expanded GridSearch + Results Table
# =====================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, log_loss, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# ===============================
# 1️⃣ Feature Engineering
# ===============================
df_balanced['vote_count_x_vote_avg'] = df_balanced['vote_count_group'] * df_balanced['vote_average_group']
df_balanced['episodes_per_season'] = df_balanced['number_of_episodes_clean'] / (df_balanced['number_of_seasons_clean'] + 1)
df_balanced['age'] = df_balanced['last_year'] - df_balanced['first_year']

# ===============================
# 2️⃣ Feature Selection
# ===============================
exclude_cols = ['popularity', 'popularity_bin', 'popularity_bin_equal', 'popularity_3cat']
feature_columns = [c for c in df_balanced.columns if c not in exclude_cols]
X = df_balanced[feature_columns]
y = df_balanced['popularity_3cat']

# ===============================
# 3️⃣ Encoding & Imputation
# ===============================
le = LabelEncoder()
y_encoded = le.fit_transform(y)

imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# ===============================
# 4️⃣ Train/Test Split
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# ===============================
# 5️⃣ RandomForest GridSearch (Expanded)
# ===============================
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'class_weight': ['balanced']
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
rf_grid.fit(X_train, y_train)

print("=== Best RandomForest ===")
print("Best params:", rf_grid.best_params_)
y_pred_rf = rf_grid.predict(X_test)
y_proba_rf = rf_grid.predict_proba(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# ===============================
# 6️⃣ XGBoost GridSearch (Expanded)
# ===============================
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 1]
}

xgb_grid = GridSearchCV(
    xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(le.classes_),
        use_label_encoder=False,
        eval_metric='mlogloss',
        n_jobs=-1,
        random_state=42
    ),
    xgb_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
xgb_grid.fit(X_train, y_train)

print("\n=== Best XGBoost ===")
print("Best params:", xgb_grid.best_params_)
y_pred_xgb = xgb_grid.predict(X_test)
y_proba_xgb = xgb_grid.predict_proba(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

# ===============================
# 7️⃣ Summary Table (Results)
# ===============================
results = []

# --- Random Forest metrics ---
report_rf = classification_report(y_test, y_pred_rf, output_dict=True, zero_division=0)
results.append({
    'Model': 'RandomForest',
    'Accuracy': round(report_rf['accuracy'], 6),
    'Precision': round(report_rf['macro avg']['precision'], 6),
    'Recall': round(report_rf['macro avg']['recall'], 6),
    'f1-score': round(report_rf['macro avg']['f1-score'], 6),
    'Log-loss': round(log_loss(y_test, y_proba_rf), 6),
    'AUC': round(roc_auc_score(pd.get_dummies(y_test), y_proba_rf, multi_class='ovr'), 6)
})

# --- XGBoost metrics ---
report_xgb = classification_report(y_test, y_pred_xgb, output_dict=True, zero_division=0)
results.append({
    'Model': 'XGBoost',
    'Accuracy': round(report_xgb['accuracy'], 6),
    'Precision': round(report_xgb['macro avg']['precision'], 6),
    'Recall': round(report_xgb['macro avg']['recall'], 6),
    'f1-score': round(report_xgb['macro avg']['f1-score'], 6),
    'Log-loss': round(log_loss(y_test, y_proba_xgb), 6),
    'AUC': round(roc_auc_score(pd.get_dummies(y_test), y_proba_xgb, multi_class='ovr'), 6)
})

results_df = pd.DataFrame(results)
print("\n=== Summary Table ===")
print(results_df)

# ===============================
# 8️⃣ Visualization (Bar Plot)
# ===============================
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'f1-score', 'AUC']

plt.figure(figsize=(10, 6))
results_melted = results_df.melt(id_vars='Model', value_vars=metrics_to_plot,
                                 var_name='Metric', value_name='Score')
sns.barplot(data=results_melted, x='Metric', y='Score', hue='Model')
plt.title('Model Performance Comparison', fontsize=14, weight='bold')
plt.ylim(0, 1)
plt.legend(title='Model')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ===============================
# 🎯 Feature Importance – RandomForest
# ===============================
best_rf = rf_grid.best_estimator_

rf_importances = pd.Series(
    best_rf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
rf_importances.head(20).plot(kind='barh')
plt.gca().invert_yaxis()
plt.title("Top 20 Feature Importances - Random Forest", fontsize=14)
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

print("\n=== RandomForest Feature Importance ===")
print(rf_importances.head(20))

# ===============================
# 🎯 Feature Importance – XGBoost
# ===============================
best_xgb = xgb_grid.best_estimator_

xgb_importances = pd.Series(
    best_xgb.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
xgb_importances.head(20).plot(kind='barh', color='orange')
plt.gca().invert_yaxis()
plt.title("Top 20 Feature Importances - XGBoost", fontsize=14)
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

print("\n=== XGBoost Feature Importance ===")
print(xgb_importances.head(20))


In [ ]:
# For GitHub upload
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import nbformat

# Path ל-notebook הנוכחי
path = '/content/drive/MyDrive/Project TV show popularity/advance project/More advanced project/For GitHub/Upload to GitHub/GitHub_3_and_4_TV_show_popularity_part_three_and_four_upload.ipynb'  # שנה לפי הנתיב שלך


# קריאה ועריכה של ה-notebook
nb = nbformat.read(path, as_version=4)

# ניקוי metadata בעייתית
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]
if "colab" in nb.metadata:
    del nb.metadata["colab"]
if "celltoolbar" in nb.metadata:
    del nb.metadata["celltoolbar"]

# שמירה מחדש
nbformat.write(nb, path)
print("✅ Notebook cleaned and ready for GitHub!")

✅ Notebook cleaned and ready for GitHub!
